# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 

data = [X,y]

  
# metadata 
print(bank_marketing.metadata) 
  
# variable information 
print(bank_marketing.variables) 

{'uci_id': 222, 'name': 'Bank Marketing', 'repository_url': 'https://archive.ics.uci.edu/dataset/222/bank+marketing', 'data_url': 'https://archive.ics.uci.edu/static/public/222/data.csv', 'abstract': 'The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 45211, 'num_features': 16, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Occupation', 'Marital Status', 'Education Level'], 'target_col': ['y'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2014, 'last_updated': 'Fri Aug 18 2023', 'dataset_doi': '10.24432/C5K306', 'creators': ['S. Moro', 'P. Rita', 'P. Cortez'], 'intro_paper': {'title': 'A data-driven approach to predict the success of bank telemarketing'

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [3]:
data = pd.concat([X,y],axis=1)


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [4]:
from ydata_profiling import ProfileReport
#ydata_profiling provides summary analysis on all features and the target
profile = ProfileReport(data, title="Practical Application III First Look Analysis")
profile.to_file("FirstLookAnalysis.html")

c:\Users\JosephCanlas\OneDrive - Aireon LLC\Documents\ML_AI Class\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 28.65it/s]


In [5]:
#Education missing values but is highly correlated with Job so I consider dropping it
#drop contact and poutcome due to amount of missing
#previous column highly skewed scaler preprocessing may help
#After dropping columns, drop na rows
data = data.drop(['education','contact','poutcome'], axis=1)  # Assuming 'Churn' is the target column
data = data.dropna()
#Split Data into Data and target


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report,precision_recall_curve
import time
import numpy as np




In [7]:
#Preprocessor
#Standardscale numeric features
#one-hot encode categorical features
X = data.drop(['y'],axis=1)
y = data['y']
#Gather features for preprocessing
non_numeric_columns = X.select_dtypes(exclude='number').columns
numeric_columns = X.select_dtypes(include='number').columns


# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_trans', StandardScaler(), numeric_columns),
        ('cat_trans', OneHotEncoder(), non_numeric_columns)
    ],remainder='passthrough')



### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [8]:
#Transform data and split training and test data
X_trans = preprocessor.fit_transform(X)
X_trans = pd.DataFrame(X_trans)
feature_names = preprocessor.get_feature_names_out()
X_trans.columns = feature_names
le = LabelEncoder()
y  = le.fit_transform(y)


In [9]:

X_train, X_test, y_train, y_test = train_test_split(X_trans, y, test_size=0.2, random_state=42)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [10]:
#Build a simple classification dummy 
dummy_clf = DummyClassifier(strategy='stratified', random_state=42)

# Fit the baseline classifier on the training data
dummy_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = dummy_clf.predict(X_test)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Baseline Classifier Accuracy:", accuracy)
print("Classification Report:")
print(report)

Baseline Classifier Accuracy: 0.7924318308291597
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      7942
           1       0.09      0.09      0.09      1043

    accuracy                           0.79      8985
   macro avg       0.49      0.49      0.49      8985
weighted avg       0.79      0.79      0.79      8985



### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
#I combine the previous Problems 8-9 in this section


In [11]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}
    
results_base = []
for name, model in models.items():
    print(f"Training {name}...")
    st = time.time()
    model.fit(X_train, y_train)
    endtime = time.time()
    print(f"Total Runtime {endtime-st}")
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    results_base.append({
        'Model': name,
        'Training ACC': train_acc,  # Convert the negative RMSE back to positive
        'Test ACC': (test_acc),  # Convert the negative RMSE back to positive
        'Runtime' : (endtime-st)
    })

Training Logistic Regression...
Total Runtime 0.3784775733947754
Training Decision Tree...
Total Runtime 0.9405145645141602
Training KNN...
Total Runtime 0.010970115661621094
Training SVM...
Total Runtime 47.24502086639404


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier()
gb_model.fit(X_trans, y)
feature_importances = gb_model.feature_importances_



In [14]:
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)
 

In [15]:
plt.figure(figsize=(10, 6))
plt.bar(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.savefig('Feature_importance.png')

In [16]:
#make a subset of X_trans_data and only use the best 10 features
features_top10 = feature_importance_df.loc[0:10,'Feature'].T.to_list()
X_feat_train = X_train[features_top10]
X_feat_test = X_test[features_top10]


In [17]:
# loop through models and obtain results
# Defined the models and their corresponding hyperparameter grids
# SVC had a rather long run time using the top five features but did not result with the best accuracy results.
# Models were reran using top 10 most important features
# Run time was cut in half with only a slight decrease in accuracy for all models
# ROC curve does not have a noticeable change so the 10 most important features determined with GBoost feature importance
# proved to help the model and input data with regards to runtime but less on actual performance metric

models = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.01, 0.1, 1, 10],
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [2, 4, 6, 8, 10],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
        }
    }
}

    # 'SVM': {
    #     'model': SVC(),
    #     'params': {
    #         'C': [0.1, 1, 10],
    #         'kernel': ['linear', 'rbf']
    #     }
    # }


In [60]:
results_allcv = []
for model_name, model_info in models.items():
    print(f"Training {model_name}...")
    clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, scoring = 'accuracy')
    st = time.time()
    clf.fit(X_train, y_train)
    endtime = time.time()
    print(f"Best parameters for {model_name}: {clf.best_params_}")
    best_score = clf.best_score_
    y_pred_test = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    print(f"Test Accuracy for {model_name}: {test_acc}")
    print(f"Best Score for {model_name}: {best_score}")
    # Append the results (model name, best parameters, training RMSE, test RMSE) to the results list
    results_allcv.append({
        'Model': model_name,
        'Training ACC': train_acc,  
        'Test ACC': test_acc, 
        'best_score': clf.best_score_,
        'Runtime' : (endtime-st)
    })
    print(f"Total Runtime {endtime-st}")

Training Logistic Regression...
Best parameters for Logistic Regression: {'C': 10}
Test Accuracy for Logistic Regression: 0.8958263772954925
Best Score for Logistic Regression: 0.8920363190540888
Total Runtime 4.742437362670898
Training Decision Tree...
Best parameters for Decision Tree: {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 10}
Test Accuracy for Decision Tree: 0.8999443516972733
Best Score for Decision Tree: 0.8956535555177185
Total Runtime 90.9932930469513
Training KNN...
Best parameters for KNN: {'n_neighbors': 7, 'weights': 'uniform'}
Test Accuracy for KNN: 0.8946021146355037
Best Score for KNN: 0.8919527265848751
Total Runtime 48.9328191280365


In [18]:
results_gridcv10 = []
for model_name, model_info in models.items():
    print(f"Training {model_name}...")
    clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, scoring = 'accuracy')
    st = time.time()
    clf.fit(X_feat_train, y_train)
    endtime = time.time()
    print(f"Best parameters for {model_name}: {clf.best_params_}")
    best_params = clf.best_params_
    best_score = clf.best_score_
    y_pred_test = clf.predict(X_feat_test)
    y_pred_train = clf.predict(X_feat_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    train_acc = accuracy_score(y_train, y_pred_train)
    
    # Append the results (model name, best parameters, training RMSE, test RMSE) to the results list
    results_gridcv10.append({
        'Model': model_name,
        'Best Parameters': best_params,
        'Training ACC': train_acc,  
        'Test ACC': test_acc, 
        'Runtime' : (endtime-st)
    })
    print(f"Total Runtime {endtime-st}")

Training Logistic Regression...
Best parameters for Logistic Regression: {'C': 1}
Total Runtime 2.083726167678833
Training Decision Tree...
Best parameters for Decision Tree: {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 10}
Total Runtime 43.802061319351196
Training KNN...
Best parameters for KNN: {'n_neighbors': 9, 'weights': 'uniform'}
Total Runtime 43.12464642524719


In [45]:
finmodel = DecisionTreeClassifier(criterion='gini',max_depth=8,min_samples_leaf=1,min_samples_split=2)
finmodel.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8)

In [46]:
from sklearn.metrics import roc_curve, auc

y_prob = finmodel.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

In [33]:
# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('ROC_curve_10features.png')

In [48]:
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('Confusion_Matrix.png')

In [49]:
precision, recall, _ = precision_recall_curve(y_test, y_prob)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.savefig('PR_curve.png')

##### Questions